In [ ]:
#https://huggingface.co/
!pip install transformers[torch]
!pip install tiktoken
!wget -O ./sample_data/crepusculoDosIdolos.txt https://raw.githubusercontent.com/mfmarlonferrari/NietzscheLLM/main/crepusculoDosIdolos.txt

--2024-06-24 19:52:02--  https://raw.githubusercontent.com/mfmarlonferrari/NietzscheLLM/main/crepusculoDosIdolos.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162098 (158K) [text/plain]
Saving to: ‘./sample_data/crepusculoDosIdolos.txt’

./sample_data/crepu 100%[===================>] 158.30K  --.-KB/s    in 0.02s   

2024-06-24 19:52:02 (7.68 MB/s) - ‘./sample_data/crepusculoDosIdolos.txt’ saved [162098/162098]



In [ ]:
PATH = './sample_data/'
dados_treino = 'conversas.txt'

In [ ]:
Continua = True
while Continua:
  texto = input("")
  if texto.lower() == 'exit':
    Continua = False
    break
  dados_treino = texto


In [ ]:
from tokenizers import ByteLevelBPETokenizer

# Initialize a ByteLevelBPETokenizer
tokenizer = ByteLevelBPETokenizer()
tokenizer.train(files=[PATH+dados_treino], vocab_size=52_000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

In [ ]:
#neste momento, nosso modelo já possui um tokenizer construído a partir dos dados
#vocab.json, lista dos tokens ordenados por frequência - converte tokens para IDs
#merges.txt - mapeia textos para tokens
#Salvando o Tokenizer
!rm -r ./sample_data/RAW_MODEL
!mkdir ./sample_data/RAW_MODEL
tokenizer.save_model(PATH+'RAW_MODEL')

['./sample_data/RAW_MODEL/vocab.json', './sample_data/RAW_MODEL/merges.txt']

In [ ]:
#Usando nosso tokenizer
#https://huggingface.co/docs/transformers/tokenizer_summary

from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained(PATH+'RAW_MODEL', max_len=512)

In [ ]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=52_000,
    max_position_embeddings=512,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

from transformers import RobertaForMaskedLM
model = RobertaForMaskedLM(config=config)

In [ ]:
# quantidade de parametros na rede neural
model.num_parameters()

83502880

In [ ]:
#carregando arquivo bruto como um dataset
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path=PATH+dados_treino,
    block_size=128,
)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:119: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [ ]:
tokenizer.decode(dataset.examples[7]['input_ids'])

'<s>Ainda não. Estou pensando em pedir alguma coisa. Sugestões?</s>'

In [ ]:
'''
Data Collators são estratégias de se construir lotes de dados
para treinar o modelo. Cria listas de amostras a partir do dataset e
permite que o Pytorch aplique a backpropagation adequadamente.
Probability = probabilidade de mascarar tokens da entrada
'''
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.1
)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir=PATH+'RAW_MODEL',
    overwrite_output_dir=True,
    num_train_epochs=1200,
    per_device_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

In [ ]:
trainer.train()

Step,Training Loss
500,2.972500
1000,1.006100
1500,0.609500
2000,0.376000
2500,0.229000
3000,0.161000
3500,0.122500
4000,0.108700
4500,0.098200
5000,0.091700


TrainOutput(global_step=16800, training_loss=0.22181900626137144, metrics={'train_runtime': 3674.9118, 'train_samples_per_second': 279.843, 'train_steps_per_second': 4.572, 'total_flos': 5561391546387456.0, 'train_loss': 0.22181900626137144, 'epoch': 1200.0})

In [ ]:
trainer.save_model(PATH+'RAW_MODEL')

In [ ]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model=PATH+'RAW_MODEL',
    tokenizer=PATH+'RAW_MODEL'
)

In [ ]:
texto = 'Você tem <mask>'
fill_mask(texto)

[{'score': 0.3114980161190033,
  'token': 325,
  'token_str': ' ir',
  'sequence': 'Você tem ir'},
 {'score': 0.2317759394645691,
  'token': 35,
  'token_str': '?',
  'sequence': 'Você tem?'},
 {'score': 0.17899788916110992,
  'token': 803,
  'token_str': ' planos',
  'sequence': 'Você tem planos'},
 {'score': 0.08394808322191238,
  'token': 293,
  'token_str': ' de',
  'sequence': 'Você tem de'},
 {'score': 0.021907635033130646,
  'token': 367,
  'token_str': ' foi',
  'sequence': 'Você tem foi'}]

In [ ]:
texto = 'Vamos <mask>'
fill_mask(texto)

[{'score': 0.9588009715080261,
  'token': 35,
  'token_str': '?',
  'sequence': 'Vamos?'},
 {'score': 0.017640303820371628,
  'token': 340,
  'token_str': ' ao',
  'sequence': 'Vamos ao'},
 {'score': 0.013531574048101902,
  'token': 17,
  'token_str': '-',
  'sequence': 'Vamos-'},
 {'score': 0.0024451706558465958,
  'token': 351,
  'token_str': ' fazer',
  'sequence': 'Vamos fazer'},
 {'score': 0.0004329589137341827,
  'token': 304,
  'token_str': ' não',
  'sequence': 'Vamos não'}]

In [ ]:
texto = 'Como <mask>'
fill_mask(texto)

[{'score': 0.45190131664276123,
  'token': 398,
  'token_str': ' está',
  'sequence': 'Como está'},
 {'score': 0.2159992903470993,
  'token': 17,
  'token_str': '-',
  'sequence': 'Como-'},
 {'score': 0.12452341616153717,
  'token': 35,
  'token_str': '?',
  'sequence': 'Como?'},
 {'score': 0.0833558663725853,
  'token': 304,
  'token_str': ' não',
  'sequence': 'Como não'},
 {'score': 0.029994778335094452,
  'token': 367,
  'token_str': ' foi',
  'sequence': 'Como foi'}]